In [11]:
import os
import re

function_name = "sum"

c_code = """
#include <stdint.h>
uint32_t sum( uint32_t *buf, uint32_t len )
{
    uint32_t acc = 0;
    uint32_t i = 0;
    for( i = 0 ; i < len ; i++ )
    {
        acc += buf[i];
    }
    return acc;
}
"""

base_code = """
int main( void )
{
    return 0;
}

void _exit( int p )
{
    while(1);
}

"""

fl_name = "c_code.c"
fl_name_base = fl_name.split(".")[0]
with open( fl_name, "w" ) as fl:
    fl.write( c_code )
    fl.write( base_code )

options = "-O3"
_cpu = "-mcpu=cortex-m0plus -mfloat-abi=soft -mthumb"
cpu = "-mcpu=cortex-m4 -mfpu=fpv4-sp-d16 -mfloat-abi=hard -mthumb"
_cpu = "-mcpu=cortex-m7 -mtune=cortex-m7 -mfpu=fpv5-sp-d16 -mfloat-abi=hard -mthumb"
_cpu = "-mcpu=cortex-m7 -mtune=cortex-m7 -mfpu=fpv5-d16 -mfloat-abi=hard -mthumb"
_cpu = "-mthumb -mfpu=fpv4-sp-d16 -mfloat-abi=hard -mtune=cortex-m7 -mcpu=cortex-m7 -fsingle-precision-constant"

cmd = "arm-none-eabi-gcc {0}.c -g3 {1} {2} -o {0}.o".format( fl_name_base, options, cpu )
print( "cmd:", cmd )
result = os.system( cmd )
if( result ):
    print( "result", result )
    raise None

cmd = "arm-none-eabi-objdump -h -S  {0}.o  > {0}.list".format( fl_name_base )
print( "cmd:", cmd )
result = os.system( cmd )
print( "result", result )
if( result ):
    print( "result", result )
    raise None

print("")

argc = 0
for line in c_code.split("\n"):
    if( function_name in line ):
        print( "#" + line )
        argc = line.split("(")[1].split(")")[0].count(",") + 1
        break

with open( fl_name_base+".list" ) as fl:
    state = 0
    for line in fl.readlines():
        line = line[:-1]
        if( state == 0 ):
            if( "<" + function_name + ">:" in line ):
                #print( "#" + line )
                state = 1
                print("""@micropython.asm_thumb
def {}( {} ):
    b(end)
    label(start)""".format( function_name, ", ".join(["r"+str(i) for i in range(argc)]) ))
        elif( state == 1 ):
            if( "__aeabi" in line ):
                print( line )
                raise None
            
            if( len(line) == 0 ):
                pass
            
            elif( line[0] == "#" ):
                print( "    #" + line )
                pass
            
            elif( len(line) > 8 and line[8] == ":" ):
                print( "    data( 2, 0x{0} ) # {1}".format( line[10:14].upper(), line[21:] ) )
                if( line[15:19] != "    " ):
                    print( "    data( 2, 0x{0} ) # {1}".format( line[15:19].upper(), line[21:] ) )
            elif( len(line) > 9 and line[9] == "<" ):
                state = 2
            
            else:
                print( "    #" + line )
                pass
        elif( state == 2 ):
            pass

print("""    label(end)
    bl(start)""")


cmd: arm-none-eabi-gcc c_code.c -g3 -O3 -mcpu=cortex-m4 -mfpu=fpv4-sp-d16 -mfloat-abi=hard -mthumb -o c_code.o
cmd: arm-none-eabi-objdump -h -S  c_code.o  > c_code.list
result 0

#uint32_t sum( uint32_t *buf, uint32_t len )
@micropython.asm_thumb
def sum( r0, r1 ):
    b(end)
    label(start)
    #    for( i = 0 ; i < len ; i++ )
    data( 2, 0xB149 ) # cbz	r1, 8116 <sum+0x16>
    data( 2, 0x1F03 ) # subs	r3, r0, #4
    data( 2, 0xEB03 ) # add.w	r1, r3, r1, lsl #2
    data( 2, 0x0181 ) # add.w	r1, r3, r1, lsl #2
    #    uint32_t acc = 0;
    data( 2, 0x2000 ) # movs	r0, #0
    #        acc += buf[i];
    data( 2, 0xF853 ) # ldr.w	r2, [r3, #4]!
    data( 2, 0x2F04 ) # ldr.w	r2, [r3, #4]!
    #    for( i = 0 ; i < len ; i++ )
    data( 2, 0x428B ) # cmp	r3, r1
    #        acc += buf[i];
    data( 2, 0x4410 ) # add	r0, r2
    #    for( i = 0 ; i < len ; i++ )
    data( 2, 0xD1FA ) # bne.n	810a <sum+0xa>
    data( 2, 0x4770 ) # bx	lr
    #    uint32_t acc = 0;
    data( 2, 0x4608 ) # mov